# Calculate sidewalk width
Using centerline extraction (also called skeleton line, axis line, or medial line extraction)

In [ ]:
# Add project src to path.
import set_path

import numpy as np
import pathlib
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
from centerline.geometry import Centerline
import shapely.ops as so

import upcp.utils.bgt_utils as bgt_utils
import upcp.utils.las_utils as las_utils

import upc_sw.poly_utils as poly_utils

In [ ]:
# Paths
pc_data_folder = '../datasets/pointclouds/'
out_folder = '../datasets/output/'

# A CRS tells Python how those coordinates relate to places on the Earth. Rijksdriehoek = epsg:28992
CRS = 'epsg:28992' #local crs

# Tolerance for centerline simplification
simplify_tolerance = 0.2

# Min area size of a sidewalk polygon in meters for which width will be computed
min_area_size = 5

# Max linestring length in meters
max_line_length = 5

# Resolution (in m) for min and avg width computation
width_resolution = 1

# Precision (in decimals) for min and avg width computation
width_precision = 1

## Read the sidewalk and obstacle data

In [ ]:
# Read sidewalk with obstacle data
df = gpd.read_file('../datasets/shp_files/sidewalks_with_obstacles.shp', crs=CRS)

## Merge the sidewalk polygons

In [ ]:
df = gpd.GeoDataFrame(geometry=gpd.GeoSeries([geom for geom in df.unary_union.geoms]), crs=CRS)
df = gpd.GeoDataFrame(df.geometry.explode(index_parts=True), crs=CRS)

In [ ]:
# Ignore sidewalk polygons that are too small
df = df[df.geometry.area > min_area_size]

## Calculate the centerlines

In [ ]:
def get_centerlines(row_geometry):
    ''' Save a NaN value when centerline calculation fails. '''
    try:
        x = Centerline(row_geometry)
    except Exception as e:
        print(e) # TODO also print rows.name[0] ??
        x = np.nan
    return x

In [ ]:
# if you get an error here, make sure you use tqdm>=4.61.2
df['centerlines'] = df.progress_apply(
                        lambda row: get_centerlines(row.geometry), axis=1)
df = df.set_geometry('centerlines')

## Remove short line ends and dead-ends

In [ ]:
df['centerlines'] = df['centerlines'].progress_apply(so.linemerge)

In [ ]:
df['centerlines'] = df['centerlines'].progress_apply(
                        lambda x: poly_utils.remove_short_lines(x, max_line_length))

## Get sidewalk widths

In [ ]:
df['centerlines'] = df['centerlines'].progress_apply(
                        lambda row: row.simplify(simplify_tolerance, preserve_topology=True))

In [ ]:
df['segments'] = df['centerlines'].progress_apply(poly_utils.get_segments)

In [ ]:
df[['avg_width', 'min_width']] = df.progress_apply(
                        lambda row: poly_utils.get_avg_width(row, width_resolution, width_precision), axis=1)

In [ ]:
# TODO: keep sidewalk ID along with segments for back-reference
segment_gdf = pd.concat([gpd.GeoDataFrame({'geometry': row.segments,
                                           'avg_width': row.avg_width,
                                           'min_width': row.min_width})
                         for _, row in df.iterrows()])
segment_gdf.set_crs(crs=CRS, inplace=True);

## Check coverage of point cloud data on sidewalks

In [ ]:
all_tiles = las_utils.get_tilecodes_from_folder(f'{pc_data_folder}run1/')
all_tiles_poly = so.unary_union([poly_utils.tilecode_to_poly(tile) for tile in all_tiles])

segment_gdf['pc_coverage'] = segment_gdf.intersects(all_tiles_poly)

## Store output

In [ ]:
pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)

segments_file = f'{out_folder}sidewalk_segments.gpkg'
segment_gdf.to_file(segments_file, driver='GPKG')